# Bounds on the Secrecy Outage Probability

## Rayleigh Fading

Text about it...

In [1]:
import numpy as np
from scipy import optimize
%matplotlib notebook
import matplotlib.pyplot as plt
from ipywidgets import interact

In [2]:
def cdf_x(x, lam=1):
    return np.maximum(1.-np.exp(-x*lam), 0)

def cdf_y(y, lam=1):
    return np.minimum(np.exp(y*lam), 1)

def w_copula(a, b):
    return np.maximum(a + b - 1, 0)

In [4]:
def lower_bound_eps_c(r_s, r_c, lam_x=1, lam_y=2, snr_bob=1, snr_eve=1):
    s = 2**r_s - 1.
    return 1.-np.exp(-s*lam_x/snr_bob)

#TODO
def upper_bound_eps_c(s, rate, lam_x=1, lam_y=2, snr_bob=1, snr_eve=1):
    alpha = 2**rate - 1.
    lam_x_tilde = alpha*lam_x
    a = 1./lam_x_tilde
    b = 1./lam_y
    if b < a:
        x_opt = (a*b*(np.log(a)-np.log(b))+a*s)/(a-b)
        y_opt = s - x_opt
        fx = 1. - np.exp(-x_opt/a)
        fy = np.exp(y_opt/b)
        c = fx + fy
    else:
        c = 1
    return c

In [6]:
def calc_outage(bound, r_s, r_c, lam_x=1., lam_y=1., snr_bob=1, snr_eve=1):
    if bound.startswith("ind"):
        func = NotImplemented #np.vectorize(cdf_laplace)
    elif bound.startswith('min') or bound.startswith("low"):
        func = np.vectorize(lower_bound_eps_c)    
    elif bound.startswith('max') or bound.startswith("up"):
        func = np.vectorize(upper_bound_eps_c)
    return func(r_s, r_c, lam_x=lam_x, lam_y=lam_y, snr_bob=snr_bob, snr_eve=snr_eve)

In [7]:
def copula_lower(a, b, r_s=1, r_c=1, lam_x=1, lam_y=1, snr_bob=1, snr_eve=1):
    t = lower_bound_eps_c(r_s, r_c, lam_x=lam_x, lam_y=lam_y, snr_bob=snr_bobr, snr_eve=snr_eve)
    c = np.minimum(a, b)
    idx_t = np.where(np.logical_and(a >= t, b >= t))
    c[idx_t] = np.maximum(a[idx_t] + b[idx_t] - 1, t)
    return c

def copula_upper(a, b, r_s=1, r_c=1, lam_x=1, lam_y=1, snr_bob=1, snr_eve=1):
    r = upper_bound_eps_c(r_s, r_c, lam_x=lam_x, lam_y=lam_y, snr_bob=snr_bob, snr_eve=snr_eve)
    c = np.minimum(a, b)
    idx_r = np.where(np.logical_and(a <= r, b <= r))
    c[idx_r] = np.maximum(a[idx_r] + b[idx_r] - r, 0)
    return c